In [1]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as ipw
import json
import subprocess
import atexit
import os
import shutil
from src import widgets, ai_agent, utils

In [2]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
GOOGLE_API_KEY = utils.read_json("/home/jovyan/gemini_api.json")

In [3]:
openbis_agent = ai_agent.OpenBISAgent(
    google_api_key=GOOGLE_API_KEY["api_key"]
)

In [4]:
# x = openbis_agent.ask_question("Which experiments are available? Give me a summary of them")
# x = openbis_agent.ask_question("Is there an experiment that uses molecule 704a? If yes, give me a summary of it including the name of sample that was used and the sample that was produced by each process step and a small description of the actions and observables.")
# x = openbis_agent.ask_question("Is there molecule 704a?")
# x = openbis_agent.ask_question("Is there a simulation in openBIS? If yes give me a summary")
# x = openbis_agent.ask_question("Considering the geometry optimisation, what was the used molecule?")
# x = openbis_agent.ask_question("Find me all the objects with type MOLECULE in openBIS")
# x = openbis_agent.ask_question("Is there substance with empa number 704 and batch a? If yes give me a summary of it")
# x = openbis_agent.ask_question("What is the molecule?")
x = openbis_agent.ask_question("Given substances with empa number 704 and batch a, how many preparations there are in openBIS?")
# x = openbis_agent.ask_question("Yes. No filters give me all that you find")

In [ ]:
# x = openbis_agent.ask_question("Ok, I see that you have a deposition process step. Tell me more about that. What substance was used?")

In [5]:
x

[HumanMessage(content='Given substances with empa number 704 and batch a, how many preparations there are in openBIS?', additional_kwargs={}, response_metadata={}, id='66aa9480-5382-4998-9d9b-9256bf90fe3d'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_openbis_objects', 'arguments': '{"type": "SUBSTANCE", "filters": {"batch": "a", "empa_number": "704"}}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--bdbfb391-7c24-4253-bacd-a12b0c91fd8e-0', tool_calls=[{'name': 'get_openbis_objects', 'args': {'type': 'SUBSTANCE', 'filters': {'batch': 'a', 'empa_number': '704'}}, 'id': '253906f9-5f65-4c7a-9cff-44d5dd4163f4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 432, 'output_tokens': 39, 'total_tokens': 471, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"name": "TUD-156", "molecules": ["20250717093148606-211"], "empa_number": "704", "batch": "a"

In [6]:
print("Question:", x[0].content)
print("Answer:", x[-1].content)

Question: Given substances with empa number 704 and batch a, how many preparations there are in openBIS?
Answer: There is 1 preparation with empa number 704 and batch a.


In [ ]:
# def get_weather(city: str) -> str:  
#     """Get weather for a given city."""
#     return f"It's always sunny in {city}!"

# from langchain_google_genai import ChatGoogleGenerativeAI
# from pydantic import BaseModel
# from langgraph.prebuilt import create_react_agent

# class ActionResponse(BaseModel):
#     type: str
#     name: str
#     duration: str
#     target_temperature: str
#     component_name: str
#     component_settings: str
    

# llm_model = ChatGoogleGenerativeAI(model = "models/gemini-2.5-flash",google_api_key = GOOGLE_API_KEY["api_key"])

# agent = create_react_agent(
#     model=llm_model,
#     tools=[get_weather],
#     response_format=ActionResponse
# )

# response = agent.invoke(
#     {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
# )

# response["structured_response"]

In [ ]:
l = []
for i in OPENBIS_SESSION.get_object_types():
    l.append(i.permId)

', '.join(l)

In [ ]:
x = OPENBIS_SESSION.get_objects(type = "PROCESS_STEP")
x[0].get_parents()